In [1]:
!ls

sample_data


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd 'drive/MyDrive/TA-10/'

/content/drive/MyDrive/TA-10


In [4]:
!pip install colabcode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 45 kB 3.8 MB/s 
     |████████████████████████████████| 8.3 MB 42.7 MB/s 
     |████████████████████████████████| 745 kB 39.2 MB/s 
     |████████████████████████████████| 428 kB 48.7 MB/s 
     |████████████████████████████████| 54 kB 3.4 MB/s 
     |████████████████████████████████| 344 kB 47.2 MB/s 
     |████████████████████████████████| 58 kB 6.4 MB/s 
     |████████████████████████████████| 561 kB 53.5 MB/s 
     |████████████████████████████████| 54 kB 771 kB/s 
     |████████████████████████████████| 132 kB 53.9 MB/s 
     |████████████████████████████████| 80 kB 5.3 MB/s 
     |████████████████████████████████| 131 kB 59.7 MB/s 
     |████████████████████████████████| 131 kB 60.6 MB/s 
     |████████████████████████████████| 130 kB 60.1 MB/s 
     |████████████████████████████████| 130 kB 58.6 MB/s 
     |████████████████████████████████| 1

In [5]:
!pip install fastapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54 kB 1.2 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 


In [6]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
import tensorflow as tf
from keras.models import load_model

In [7]:
# Read X_train with Series Type
X_train = pd.read_csv('dataset/X_train.csv', index_col = 0, squeeze = True)
print(X_train)

2927     http://lindasyardarts.com/products.php?catID=9...
16152    ' )  UNION ALL SELECT NULL, NULL, NULL, NULL, ...
5659                              http://www.worldsex.com/
16250    "  )  )   )  UNION ALL SELECT NULL, NULL, NULL...
11784     You make gender assignment without surgery , ...
                               ...                        
11284     To best knowledge , however , Sunday collecti...
11964     Obama &apos;s Health Care Walk Back Amid fire...
5390                         http://www.sexytoonlinks.com/
860      /dvwa/vulnerabilities/sqli/?Submit=Submit&id=1...
15795    " UNION ALL SELECT NULL, NULL, NULL, NULL, NUL...
Name: Sentence, Length: 14440, dtype: object


In [8]:
max_words = 24098
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)

In [9]:
from pydantic import BaseModel

class Sql(BaseModel):
    url: str
    class Config:
      schema_extra = {
          "example": {
              "url": "/dvwa/login.php", 
           }
      }

In [10]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
import pickle

app = FastAPI()

origins = ["*"]
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.on_event("startup")
def greeting():
    print("Welcome To Detection SQL Injection")

@app.get('/')
def index():
    return {'message': 'This is the homepage of the API '}


@app.post('/predict')
def predict_url(data: Sql):
    received = data.dict()
    url = received['url']
    

    sample_texts_list = []
    # Change str to list type
    sample_texts_list.append(url)
    txts = tok.texts_to_sequences(sample_texts_list)
    input_val = sequence.pad_sequences(txts, maxlen=max_len)
    loaded_model = tf.keras.models.load_model('model/lstm_model.h5')
    result=loaded_model.predict(input_val)
    last_str=""
    if result > 0.5:
      last_str="sql injection"
    elif result <= 0.5:
      last_str="non sql injection"
    return {'message': last_str}

In [11]:
from colabcode import ColabCode
server = ColabCode(port=1000, code=False)

In [ ]:
server.run_app(app=app)

Public URL: NgrokTunnel: "https://cd13-34-86-224-86.ngrok.io" -> "http://localhost:1000"
